<a href="https://colab.research.google.com/github/srikanth0904/multi-class-animal-classification-/blob/main/Copy_of_major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import files
uploaded =files.upload()

Saving Emp_Train_2.csv to Emp_Train_2.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


In [3]:
df=pd.read_csv("Emp_Train_2.csv")
df

,EmployeeID,Tenure,Salary,PerformanceScore,Department,JobRole,Layoff
0,1,7,105629,4,Engineering,Manager,0
1,2,15,82603,3,Finance,Developer,0
2,3,11,111974,4,Engineering,Executive,0
3,4,8,56399,1,HR,Manager,1
4,5,7,115382,4,Sales,Developer,0
...,...,...,...,...,...,...,...
995,996,1,102240,3,Sales,Executive,0
996,997,8,94315,4,Finance,Analyst,0
997,998,7,99249,3,Sales,Consultant,0
998,999,3,100587,1,Marketing,Developer,0


In [ ]:
df.isnull().sum()

,0
EmployeeID,0
Tenure,0
Salary,0
PerformanceScore,0
Department,0
JobRole,0
Layoff,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   EmployeeID        1000 non-null   int64 
 1   Tenure            1000 non-null   int64 
 2   Salary            1000 non-null   int64 
 3   PerformanceScore  1000 non-null   int64 
 4   Department        1000 non-null   object
 5   JobRole           1000 non-null   object
 6   Layoff            1000 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 54.8+ KB


In [ ]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [ ]:
X = df.drop(['Layoff'], axis=1)
y = df['Layoff']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=2)

In [ ]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model # Import Model
from tensorflow.keras.optimizers import Adam # Import Adam

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model weights, training
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# Define the number of output classes for binary classification
num_classes = 2

predictions = Dense(num_classes, activation='softmax')(x) # output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
# Model summary
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,571,778 (13.63 MB)

 Trainable params: 1,313,794 (5.01 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Preprocessing the data.
# Define image size and batch_size - Trained model - MobileNetV2 - 224 x 224
img_size = (224, 224)
batch_size = 64 # There can be 64 images in a batch

# Data Augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

# Create training and validation generators
# Note: 'dataset_path' is not defined in this cell.
# This code is for image data and is likely not applicable to the tabular data in other cells.
# Assuming 'dataset_path' is a variable defined elsewhere pointing to a directory of images.
dataset_path = 'path/to/your/image/dataset' # Placeholder - Replace with your actual dataset path

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/image/dataset'

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# --- Code moved from cell N8H7BfKfgLoA to define necessary variables ---
# Load dataset
# Assuming the file is in the current working directory after kagglehub download
df = pd.read_csv('Emp_Train_2.csv') # Use the correct filename from the successful load in Y-y2M4uk-NIb

# Create binary target: 1 if layoffs occurred, else 0 using the correct column name
df['laid_off_flag'] = (df['Layoff'] > 0).astype(int)

# Select features (drop non-predictive or target columns using correct column names)
X = df.drop(['Layoff', 'laid_off_flag'], axis=1) # Drop 'Layoff' and the newly created 'laid_off_flag' from features

# Target variable
y = df['laid_off_flag']

# Split into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identify numeric and categorical columns
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(include='object').columns

# Impute missing values
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

X_train[num_cols] = imputer_num.fit_transform(X_train[num_cols])
X_test[num_cols] = imputer_num.transform(X_test[num_cols])

X_train[cat_cols] = imputer_cat.fit_transform(X_train[cat_cols])
X_test[cat_cols] = imputer_cat.transform(X_test[cat_cols])

# cat_features_indices are needed for CatBoost Classifier
cat_features_indices = [X_train.columns.get_loc(col) for col in cat_cols]
# --- End of moved code ---


# Train the model
# Pass the indices of the categorical columns to cat_features
model = CatBoostClassifier(cat_features=cat_features_indices, verbose=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("CatBoost Test Accuracy:", accuracy_score(y_test, y_pred))

Learning rate set to 0.009366
0:	learn: 0.6888833	total: 3.02ms	remaining: 3.02s
100:	learn: 0.5104462	total: 207ms	remaining: 1.84s
200:	learn: 0.4761721	total: 430ms	remaining: 1.71s
300:	learn: 0.4521518	total: 651ms	remaining: 1.51s
400:	learn: 0.4305748	total: 904ms	remaining: 1.35s
500:	learn: 0.4128020	total: 1.14s	remaining: 1.14s
600:	learn: 0.3939918	total: 1.41s	remaining: 938ms
700:	learn: 0.3748083	total: 1.67s	remaining: 711ms
800:	learn: 0.3542851	total: 1.94s	remaining: 481ms
900:	learn: 0.3342936	total: 2.23s	remaining: 245ms
999:	learn: 0.3166908	total: 2.52s	remaining: 0us
CatBoost Test Accuracy: 0.805
